In [1]:
import gensound_synth

In [5]:
synth = gensound_synth.Synth({0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1},'default','../presets')

In [8]:
synth._setparam('osc1_wave','square')

In [6]:
synth.generate_audio('A3',5e3).play()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
audio = synth.generate_audio('A3',5*1e3)
tmp = audio | synth.generate_audio('C4',5*1e3)
tmp.play()

In [6]:
import json
json.load(open('../presets/timbre.json'))['default']

{'lfo1_type': 'sine',
 'lfo1_freq': 0.5,
 'lfo1_depth': 0.5,
 'lfo2_type': 'sine',
 'lfo2_freq': 0.5,
 'lfo2_depth': 0.5,
 'osc1_freq': 400,
 'osc1_wave': 'square',
 'osc1_detune': 30,
 'osc2_freq': 400,
 'osc2_wave': 'triangle',
 'osc2_detune': 30,
 'n_voices': 6,
 'sub_freq': 100,
 'sub_wave': 'sine',
 'sub_detune': 30,
 'hf_noise_freq': 1000,
 'hf_noise_wave': 'sine',
 'lf_noise_freq': 100,
 'lf_noise_wave': 'sine',
 'filter1_type': 'lowpass',
 'filter1_freq': 1000,
 'filter2_type': '',
 'filter2_freq': 0,
 'filter3_type': '',
 'filter3_freq': 0,
 'reverb': 0.5,
 'vibrato_freq': 0.5,
 'vibrato_depth': 0.5,
 'attack': 500.0,
 'decay': 0.5,
 'sustain': 0.5,
 'release': 500.0,
 'fade_in': 0.5,
 'fade_out': 0.5,
 'automation': {'filter1_freq_type': 'constant',
  'filter1_freq_rate': 0,
  'filter2_freq_type': 'constant',
  'filter2_freq_rate': 0,
  'filter3_freq_type': 'constant',
  'filter3_freq_rate': 0}}

In [29]:
mellow_o = {
        'lfo1_type': 'na',
        'lfo1_freq': 0, # for focus
        'lfo1_depth': 0.25,

        'lfo2_type': 'na',
        'lfo2_freq': 0, # slow wobble
        'lfo2_depth': 0.5,
        
        'osc1_wave': 'sawtooth',
        'osc1_detune': 6,

        'osc2_wave': 'sawtooth',
        'osc2_detune': 6,
        'osc2_relative_shift': 2,

        'n_voices': 2, # too high and this introduces a phaser efect

        'sub_freq': 0,
        'sub_wave': 'sine',
        'sub_detune': 30,

        'hf_noise_type': 'na', # na/white/pink

        'lf_noise_type': 'na',

        'filter1_type': 'lowpass',
        'filter1_freq': 525,

        'filter2_type': 'highpass',
        'filter2_freq': 200,

        'filter3_type': 'na',
        'filter3_freq': 0,

        'reverb': 0.5,

        'vibrato_freq': 0,
        'vibrato_depth': 0,

        'attack': 780.0,
        'decay': 0.05,
        'sustain': 0.9,
        'release': 4890.0,

        # 'fade_in': 0.5,
        # 'fade_out': 0.5,

        'automation': {
                'filter1_freq_type': 'constant',
                'filter1_freq_rate': 0,
                'filter2_freq_type': 'constant',
                'filter2_freq_rate': 0,
                'filter3_freq_type': 'constant',
                'filter3_freq_rate': 0
                }
        }

In [4]:
db_to_gain = lambda x: 10**(x/20)

In [188]:
import os
import gensound
from gensound.signals import Sine, Sawtooth, Square, Triangle, WhiteNoise, Silence, PinkNoise, Mix, WAV
from gensound.effects import OneImpulseReverb, Vibrato
from gensound.filters import SimpleBandPass, SimpleHPF, SimpleLPF, SimpleHighShelf, SimpleLowShelf
from gensound.transforms import ADSR, Fade, Amplitude, CrossFade
from gensound.curve import SineCurve, Constant, Line

class Synth():
    def __init__(self, params):
        # self.preset = preset
        # if self.preset == 'default':
            # load json file containing default settings
        # data = json.load(open(os.path.join(path_to_preset,'timbre.json')))[self.preset]
        # data = mellow_o
        # print(data)
        self.__dict__ = params#data
        self.NOTES_FLAT = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
        self.NOTES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.filters={}
        for filter_number in ['filter1','filter2','filter3']:
            if self.__dict__[filter_number+'_type'] == 'na':
                self.__dict__[filter_number] = None
            else:
                match self.__dict__[filter_number+'_type']:
                    case 'lowpass':
                        self.filters[filter_number] = SimpleLPF
                    case 'highpass':
                        self.filters[filter_number] = SimpleHPF
                    case 'bandpass':
                        self.filters[filter_number] = SimpleBandPass
                    case 'lowshelf':
                        self.filters[filter_number] = SimpleLowShelf
                    case 'highshelf':
                        self.filters[filter_number] = SimpleHighShelf
                # self.__dict__[filter_number] = f(self.__dict__[filter_number+'_freq'])
    def _setparam(self, param, value):
        setattr(self, param, value)
    def pitch_to_freq(self, pitch):
        return 440 * 2**((pitch-69)/12)
    def shift_freq_by_cents(self, freq, cents):
        return freq*(10**(cents/(1200*3.322038403)))
    def semitones_to_cents(self, semitones):
        return semitones*100
    def cents_to_semitones(self, cents):
        return cents/100
    def octave_to_multiplier(self, octave):
        return 2**(octave)
    def NoteToMidi(self, KeyOctave):
        # KeyOctave is formatted like 'C#3'
        key = KeyOctave[:-1]  # eg C, Db
        octave = KeyOctave[-1]   # eg 3, 4
        answer = -1

        try:
            if 'b' in key:
                pos = self.NOTES_FLAT.index(key)
            else:
                pos = self.NOTES_SHARP.index(key)
        except:
            print('The key is not valid', key)
            return answer

        answer += pos + 12 * (int(octave) + 1) + 1
        return answer
    def detuned_voices(self, pitch, relative_shift, duration, detune_pct, n_voices, wave_type='square'):
        # n_voices = how many oscillators in the array
        # detune_range = the difference in cents between the highest and lowest oscillators in the array
        if isinstance(pitch, str):
            freq = self.pitch_to_freq(self.NoteToMidi(pitch))
        detune_range = freq * int(detune_pct)/100
        all_cents = [i*detune_range/n_voices - detune_range/2 for i in range(n_voices)] # how much to detune each signal in the array
        # print(isinstance(pitch, str))
        match wave_type: # REQUIRES PYTHON 3.10
            case 'sine':
                f = Sine
            case 'square':
                f = Square
            case 'triangle':
                f = Triangle
            case 'sawtooth':
                f = Sawtooth
        if isinstance(pitch, str):
            freq = self.pitch_to_freq(self.NoteToMidi(pitch))
        return gensound.mix([f(self.shift_freq_by_cents(freq*relative_shift,cents),duration) for cents in all_cents])
    def generate_audio(self, pitch,duration):
        # self._update()
        self.pitch = pitch
        # 2 LFOS for amplitude modulation
        if self.lfo1_type != 'na':
            self.lfo1 = Amplitude(SineCurve(frequency=self.lfo1_freq, depth=self.lfo1_depth, baseline=0.5, duration=duration))
        else:
            self.lfo1 = Amplitude(Constant(1, duration=duration))
        if self.lfo2_type != 'na':
            self.lfo2 = Amplitude(SineCurve(frequency=self.lfo2_freq, depth=self.lfo2_depth, baseline=0.5, duration=duration))
        else:
            self.lfo2 = Amplitude(Constant(1, duration=duration))

        # 2 oscillators for the main sound
        self.osc1 = self.detuned_voices(self.pitch, relative_shift=1, duration=duration,detune_pct = self.osc1_detune, n_voices = self.n_voices, wave_type=self.osc1_wave)
        self.osc2 = self.detuned_voices(self.pitch, relative_shift=self.osc2_relative_shift, duration=duration,detune_pct = self.osc2_detune, n_voices = self.n_voices, wave_type=self.osc2_wave)
        # 1 oscillator for the sub bass
        if self.sub_freq != 0:
            self.sub_bass = self.detuned_voices(self.sub_freq, relative_shift=1, duration=duration,detune_pct = self.sub_detune, n_voices = self.n_voices, wave_type='sine')
        else:
            self.sub_bass = Silence(duration=duration)
        # 1 oscillator for the high frequency noise
        if self.hf_noise_type != 'na':
            self.high_freq_noise = WhiteNoise(duration=duration)
        else:
            self.high_freq_noise = Silence(duration=duration)
        # 1 oscillator for the low frequency noise
        if self.lf_noise_type != 'na':
            self.low_freq_noise = PinkNoise(duration=duration)
        else:
            self.low_freq_noise = Silence(duration=duration)
            
        # Filters
        self.filter1 = self.filters['filter1'](self.filter1_freq)
        self.filter2 = self.filters['filter2'](self.filter2_freq)

        # EFFECTS
        #   - Reverb
        #   - Vibrato

        # AMPLITUDE ENVELOPES
        #   - 2 ADSR envelopes
        #   - Fade in and out

        # apply lfo1 and lfo2 to osc1 and osc2 amplitude
        if self.lfo1_freq != 0:
            # self.lfo1 = self.lfo1 * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)
            self.lfo1 = self.osc1 * self.lfo1
        if self.lfo2_freq != 0:
            # self.lfo2 = self.lfo2 * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)
            self.lfo2 = self.osc2 * self.lfo2

        self.output = self.osc1 + self.osc2 + self.sub_bass + self.high_freq_noise + self.low_freq_noise
        if duration < 2:
            self.output = self.output * Fade(is_in=True,duration=duration*1000/2)
            self.output = self.output * Fade(is_in=False,duration=duration*1000/2)
        else:
            self.output *= Fade(is_in=True, duration=2000)
            self.output *= Fade(is_in=False, duration=2000)
        # self.output *= self.filter1
        # self.output *= self.filter2
        self.output = self.output * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)
        self.output *= self.filter1
        self.output *= self.filter2
        return self.output

In [189]:
params = {'lfo1_type': 'sine', 'lfo1_freq': 0.5, 'lfo1_depth': 0.5, 'lfo2_type': 'sine', 'lfo2_freq': 0.5, 'lfo2_depth': 0.5, 'osc1_freq': 200.0, 'osc1_relative_shift': 1.0, 'osc1_wave': 'square', 'osc1_detune': 30.0, 'osc2_freq': 400.0, 'osc2_relative_shift': 1.0, 'osc2_wave': 'triangle', 'osc2_detune': 30.0, 'n_voices': 6, 'sub_freq': 100.0, 'sub_wave': 'sine', 'sub_detune': 30.0, 'hf_noise_freq': 1000.0, 'hf_noise_wave': 'sine', 'lf_noise_freq': 100.0, 'lf_noise_wave': 'sine', 'filter1_type': 'lowpass', 'filter1_freq': 1000.0, 'filter2_type': 'na', 'filter2_freq': 0.0, 'filter3_type': 'na', 'filter3_freq': 0.0, 'reverb': 0.5, 'vibrato_freq': 0.5, 'vibrato_depth': 0.5, 'attack': 500.0, 'decay': 0.5, 'sustain': 0.5, 'release': 500.0, 'fade_in': 0.5, 'fade_out': 0.5}

In [191]:
synth=Synth(params)
synth.generate_audio(42,5e3).play()

TypeError: 'int' object is not subscriptable

# Interfacing with musical parameters

In [74]:
from musical_params_unified import MusicalParameters
import numpy as np
mp = MusicalParameters(
                        duration=90, 
                        ending_chord_min_length=2, 
                        starting_key='C', 
                        starting_length="VL", 
                        starting_note_density=3, 
                        starting_octave=4, 
                        preset_dict = {
                            "default":
                                {
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                },
                            "1": 
                            {
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                },
                            "2":
                            {
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                },
                            "3":{
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                },
                            "4":{
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                },
                            "5":
                            {
                                    "composition_probabilities":[0.5,0.5], 
                                    "key_transition_probability":[0.3,0.3,0.3,0.1], 
                                    "key_type":'major', 
                                    "note_density_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]]), 
                                    "octave_transition_matrix":np.array([[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25]]), 
                                    "duration_transition_matrix":np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
                                }
                                    }
                            )


In [95]:
output = mp.get_musical_parameters(mood_map={0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1})

In [10]:
output['chords']

[array([38, 45]),
 array([54, 58, 61, 66, 49]),
 array([55, 59, 62, 67]),
 array([56, 60, 63, 68, 75]),
 array([49, 53, 56]),
 array([32, 39]),
 array([49, 53, 56, 61, 44]),
 array([49, 53, 56, 61, 68]),
 array([66, 73]),
 array([36, 40, 43, 48]),
 array([65, 69, 72]),
 array([60]),
 array([61, 68]),
 array([54, 58, 61, 66, 49]),
 array([72, 76, 79, 84]),
 array([63, 67, 70, 75, 58]),
 array([58, 62, 65]),
 array([66, 70, 73]),
 array([36, 40, 43, 48, 55]),
 array([41]),
 array([58]),
 array([60, 64, 67, 72, 79]),
 array([30, 34, 37]),
 array([31, 35, 38, 43]),
 array([32, 39]),
 array([37]),
 array([53, 60]),
 array([54, 58, 61]),
 array([58, 62, 65]),
 array([67]),
 array([37, 41, 44]),
 array([41, 44, 48]),
 array([42, 46, 49, 54, 61]),
 array([43, 47, 50, 55]),
 array([39, 43, 46, 51, 34]),
 array([58, 62, 65, 70]),
 array([30, 34, 37]),
 array([35]),
 array([36, 40, 43]),
 array([37, 44]),
 array([38, 42, 45, 50]),
 array([38]),
 array([51, 55, 58, 63]),
 array([48, 52, 55, 60, 43

In [11]:
output['durations']

[0.8,
 0.62,
 2.22,
 2.75,
 3.18,
 1.77,
 6.7,
 2.28,
 4.06,
 3.71,
 0.51,
 5.27,
 2.58,
 0.88,
 0.68,
 6.45,
 6.12,
 4.6,
 2.29,
 5.69,
 0.86,
 1.57,
 1.44,
 3.9,
 3.65,
 0.74,
 3.45,
 5.64,
 2.61,
 2.9800000000000226,
 3.6,
 2.91,
 6.32,
 2.48,
 0.74,
 7.04,
 1.9,
 3.76,
 3.32,
 4.68,
 0.85,
 1.07,
 4.17,
 1.33,
 1.96,
 0.84,
 5.95,
 2.93,
 1.46,
 1.23,
 2.29,
 1.61,
 3.72,
 4.84,
 1.72,
 2.95,
 0.82,
 7.88,
 6.87]

In [96]:
sum(output['durations'])

180.57

In [8]:
def generate_timbral_curves(mood_map, desired_duration):
    """
    Generates timbral curves for each mood in the mood map.
    """
    timbre = json.load(open('../presets/timbre.json'))
    lfo={'na':0, 'sine':1, 'sawtooth':2, 'square':3, 'triangle':4}
    osc = {'na':0, 'sine':1, 'sawtooth':2, 'square':3, 'triangle':4}
    filters = {'na':0, 'lowpass':1, 'highpass':2}
    timbral_curves = {}
    emotion_intervals = {}
    for i in range(len(mood_map.keys())-1):
        emotion_intervals[(list(mood_map.keys())[i], list(mood_map.keys())[i+1])] = (list(mood_map.values())[i], list(mood_map.values())[i+1])
    if list(mood_map.keys())[i+1] < desired_duration:
        # emotion_intervals.append((list(mood_map.keys())[i+1], desired_duration))
        emotion_intervals[(list(mood_map.keys())[i+1], desired_duration)] = (list(mood_map.values())[i+1], list(mood_map.values())[i+1])
    
    # for interval in emotion_intervals.keys():
    
    
    return emotion_intervals, timbral_curves

def generate_audio(musical_param_dict, mood_map):
    """
    Generates audio from the musical parameters
    """
    synth = gensound_synth.Synth('default','../presets')
    chords = musical_param_dict['chords']
    durations = musical_param_dict['durations']
    for i in range(len(chords)):
        if i == 0:
            audio = 0
    return

In [104]:
emotion_timestamp_dict = {0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1}
desired_duration = 90
keys = list(emotion_timestamp_dict.keys())
values = list(emotion_timestamp_dict.values())

emotion_intervals = {}
for i in range(len(emotion_timestamp_dict.keys())-1):
    emotion_intervals[(keys[i], keys[i+1])] = (values[i], values[i+1])
if keys[i+1] < desired_duration:
    # emotion_intervals.append((list(mood_map.keys())[i+1], desired_duration))
    emotion_intervals[(keys[i+1], desired_duration)] = (values[i+1], values[i+1])

In [106]:
emotion_intervals

{(0.0, 20.3): (1, 2),
 (20.3, 40.6): (2, 3),
 (40.6, 60.9): (3, 4),
 (60.9, 81.2): (4, 5),
 (81.2, 90): (5, 1)}

In [69]:
def interpolate_parameter(initial, final, num_steps):
    """
    Interpolates between two values
    """
    if type(initial) == str:
        return np.array([initial if i <= num_steps/2 else final for i in range(num_steps)])
    return np.linspace(initial, final, num_steps)


In [35]:
themes = json.load(open(os.path.join(os.getcwd(),'..','presets', 'themes.json')))
musical_params = json.load(open(os.path.join(os.getcwd(),'..','presets', 'musical_params.json')))
timbral_params = json.load(open(os.path.join(os.getcwd(),'..','presets', 'timbre.json')))

In [54]:
print(k)
print(v)

[(0.0, 20.3), (20.3, 40.6), (40.6, 60.9), (60.9, 81.2), (81.2, 90), (90, 100)]
[(1, 2), (2, 3), (3, 4), (4, 5), (5, 1), (1, 1)]


In [57]:
emotion_intervals[k[interval]]

(1, 2)

In [107]:
k = list(emotion_intervals.keys())


In [70]:
k

[(0.0, 20.3), (20.3, 40.6), (40.6, 60.9), (60.9, 81.2), (81.2, 90), (90, 100)]

In [134]:
output = {'durations': [2.51, 1.78, 2.41, 1.94, 2.76, 5.93, 2.63, 0.34000000000000385, 4.9, 7.25, 1.27, 1.41, 0.98, 0.7, 3.789999999999999, 4.99, 1.86, 0.64, 2.48, 2.59, 4.42, 0.91, 1.08, 1.3299999999999992, 1.56, 1.76, 4.55, 1.85, 7.93, 1.09, 1.560000000000004, 0.61, 0.98, 2.68, 2.19, 2.339999999999998]}

In [135]:
sum(durs)

180.67999999999998

In [138]:
k = list(emotion_intervals.keys())
v = list(emotion_intervals.values())

durs = chord_sequence['durations']
steps={}
start = 0
end = 0
for interval in k:
    # print(interval)
    while sum(durs[start:end]) < (interval[1] - interval[0]):
        # print(interval[1],sum(durs[start:end]))
        end += 1
        # print(end)
        if end > len(durs):
            break
    print(interval)
    print(start,end)
    print(sum(durs[start:end]))
    print(interval[1]-interval[0])
    print("--------------------")
    start = end 
# print(end)

(0.0, 20.3)
0 8
20.3
20.3
--------------------
(20.3, 40.6)
8 15
20.3
20.3
--------------------
(40.6, 60.9)
15 24
20.299999999999997
20.299999999999997
--------------------
(60.9, 81.2)
24 31
20.300000000000004
20.300000000000004
--------------------
(81.2, 90)
31 36
8.799999999999997
8.799999999999997
--------------------


In [133]:
sum(durs[22:29])

24.99000000000003

In [152]:
d = mp.get_musical_parameters(mood_map={0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1})
chords, durations = d['chords'], d['durations']

In [154]:
durations

[0.73,
 0.72,
 4.24,
 2.34,
 3.9,
 7.47,
 2.34,
 4.72,
 1.94,
 1.31,
 4.29,
 5.07,
 1.9,
 4.8,
 0.77,
 1.06,
 4.08,
 6.98,
 1.61,
 3.31,
 6.35,
 0.54,
 1.52,
 0.54,
 7.67,
 1.44,
 1.01,
 1.38,
 0.63,
 2.14,
 3.1999999999999797,
 1.33,
 1.02,
 6.44,
 0.79,
 0.86,
 1.71,
 7.63,
 5.45,
 3.4,
 1.63,
 1.01,
 0.9,
 0.89,
 5.98,
 0.64,
 1.48,
 3.48,
 2.5,
 0.87,
 0.98,
 1.58,
 6.59,
 0.75,
 1.16,
 3.44,
 2.28,
 5.32,
 1.3,
 2.49,
 0.63,
 2.38,
 7.06,
 7.3]

In [164]:

t_params = {}
k = list(emotion_intervals.keys())
v = list(emotion_intervals.values())
steps={}
start = 0
end = 0
for interval in k:
    while sum(durations[start:end]) < (interval[1] - interval[0]):
        end += 1
        if end > len(durations):
            break
    # ic(interval)
    # ic(start,end)
    # ic(sum(durations[start:end]))
    # ic(interval[1]-interval[0])
    # ic("--------------------")
    steps[interval] = end - start
    start = end

    duration = list(interval)[1] - list(interval)[0]
    for param in timbral_params['default'].keys():
        if param != 'automation':
            initial_param_value = timbral_params[str(emotion_intervals[interval][0])][param]
            final_param_value = timbral_params[str(emotion_intervals[interval][1])][param]
            if interval == k[0]:
                t_params[param] = interpolate_parameter(initial_param_value, final_param_value, steps[interval])
            else:
                t_params[param] = np.concatenate((t_params[param], interpolate_parameter(initial_param_value, final_param_value, steps[interval])))
    #     t_params[key] = interpolate_parameter(timbral_params[key][interval[0]], timbral_params[key][interval[1]], int(emotion_intervals[interval][1]*desired_duration))

    # for interval in range(len(k)):
    #     print(interval)
    #     # print(emotion_intervals[interval])
    #     duration = list(emotion_intervals.keys())[interval][1] - list(emotion_intervals.keys())[interval][0]
    #     initial_param_value = timbral_params[str(emotion_intervals[k[interval]][0])][param]
    #     final_param_value = timbral_params[str(emotion_intervals[k[interval]][1])][param]
    #     print(duration)

    #     if interval == 0:
    #         t_params[param] = interpolate_parameter(initial_param_value, final_param_value, steps[interval])
    #     else:
    #         t_params[param] = np.concatenate((t_params[param], interpolate_parameter(initial_param_value, final_param_value, steps[interval])))
    # #     t_params[key] = interpolate_parameter(timbral_params[key][interval[0]], timbral_params[key][interval[1]], int(emotion_intervals[interval][1]*desired_duration))

In [173]:
for i in range(len(t_params[list(t_params.keys())[0]])):
    params={}
    for param in t_params.keys():
        params[param] = t_params[param][i]

In [176]:
params = {'lfo1_type': 'sine', 'lfo1_freq': 0.5, 'lfo1_depth': 0.5, 'lfo2_type': 'sine', 'lfo2_freq': 0.5, 'lfo2_depth': 0.5, 'osc1_freq': 400.0, 'osc1_wave': 'square', 'osc1_detune': 30.0, 'osc2_freq': 400.0, 'osc2_wave': 'triangle', 'osc2_detune': 30.0, 'n_voices': 6, 'sub_freq': 100.0, 'sub_wave': 'sine', 'sub_detune': 30.0, 'hf_noise_freq': 1000.0, 'hf_noise_wave': 'sine', 'lf_noise_freq': 100.0, 'lf_noise_wave': 'sine', 'filter1_type': 'lowpass', 'filter1_freq': 1000.0, 'filter2_type': 'na', 'filter2_freq': 0.0, 'filter3_type': '', 'filter3_freq': 0.0, 'reverb': 0.5, 'vibrato_freq': 0.5, 'vibrato_depth': 0.5, 'attack': 500.0, 'decay': 0.5, 'sustain': 0.5, 'release': 500.0, 'fade_in': 0.5, 'fade_out': 0.5}

In [166]:
t_params

{'lfo1_type': array(['sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine'], dtype='<U4'),
 'lfo1_freq': array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]),
 'lfo1_depth': array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]),
 'lfo2_type': array(['sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine', 'sine',
        'sine', 'sine', 'sine', 'sine', 'sine', 

In [165]:
steps

{(0.0, 20.3): 7,
 (20.3, 40.6): 7,
 (40.6, 60.9): 7,
 (60.9, 81.2): 11,
 (81.2, 90): 4}

In [21]:
emotion_timestamp_dict = {0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1}
desired_duration = 90

In [22]:
keys = list(emotion_timestamp_dict.keys())
values = list(emotion_timestamp_dict.values())
emotion_intervals = {}
for i in range(len(emotion_timestamp_dict.keys())-1):
    emotion_intervals[(keys[i], keys[i+1])] = (values[i], values[i+1])
if keys[i+1] < desired_duration:
    # emotion_intervals.append((list(mood_map.keys())[i+1], desired_duration))
    emotion_intervals[(keys[i+1], desired_duration)] = (values[i+1], values[i+1])

In [26]:
chords, durations = mp.generate_chords(emotion_timestamp_dict)

AttributeError: 'MusicalParameters' object has no attribute 'generate_chords'

In [23]:
emotion_intervals

{(0.0, 20.3): (1, 2),
 (20.3, 40.6): (2, 3),
 (40.6, 60.9): (3, 4),
 (60.9, 81.2): (4, 5),
 (81.2, 90): (5, 1)}

In [16]:
def interpolate_parameter(initial, final, num_steps):
    """
    Interpolates between two values
    """
    if type(initial) == str:
        return np.array([initial if i <= num_steps/2 else final for i in range(num_steps)])
    return np.linspace(initial, final, num_steps)

In [18]:
import numpy as np

In [9]:
d,_=generate_timbral_curves({0.0:1, 20.3:2, 40.6:3, 60.9:4, 81.2:5, 90:1}, 100)
d

{(0.0, 20.3): (1, 2),
 (20.3, 40.6): (2, 3),
 (40.6, 60.9): (3, 4),
 (60.9, 81.2): (4, 5),
 (81.2, 90): (5, 1),
 (90, 100): (1, 1)}

In [30]:
mellow_o.keys()

dict_keys(['lfo1_type', 'lfo1_freq', 'lfo1_depth', 'lfo2_type', 'lfo2_freq', 'lfo2_depth', 'osc1_wave', 'osc1_detune', 'osc2_wave', 'osc2_detune', 'osc2_relative_shift', 'n_voices', 'sub_freq', 'sub_wave', 'sub_detune', 'hf_noise_type', 'lf_noise_type', 'filter1_type', 'filter1_freq', 'filter2_type', 'filter2_freq', 'filter3_type', 'filter3_freq', 'reverb', 'vibrato_freq', 'vibrato_depth', 'attack', 'decay', 'sustain', 'release', 'automation'])

In [5]:
from icecream import ic
import json
import gensound
from gensound.signals import Sine, Sawtooth, Square, Triangle, WhiteNoise, Silence, PinkNoise, Mix, WAV
from gensound.effects import OneImpulseReverb, Vibrato
from gensound.filters import SimpleBandPass, SimpleHPF, SimpleLPF, SimpleHighShelf, SimpleLowShelf
from gensound.transforms import ADSR, Fade, Amplitude, CrossFade
from gensound.curve import SineCurve, Constant, Line

In [10]:


curves={}
for i in range(len(d.keys())):
    interval = list(d.keys())[i]
    duration = interval[1] - interval[0]
    initial_mood = d[list(d.keys())[i]][0]
    final_mood = d[list(d.keys())[i]][1]
    if i == 0:
        for param in list(timbre['default'].keys()):
            # create a gensound line object with the starting and ending values for each parameter based on the mood found from timbre.json
            if (param[:2] == 'lfo') and (param[-5:]=='_type'):
                curves[param] = Constant(lfo[timbre[str(initial_mood)][param]], duration)
            if (param[:5] == 'filter') and (param[-5:]=='_type'):
                curves[param] = Constant(filters[timbre[str(initial_mood)][param]], duration)
            if param[-5:] == '_wave':
                curves[param] = Constant(osc[timbre[str(initial_mood)][param]], duration)
            if param == 'automation':
                pass
            # if the parameter is an lfo, use the lfo lookup dict    
            else:
                curves[param] = Line(timbre[str(initial_mood)][param], timbre[str(final_mood)][param], duration)
    else:
        for param in list(timbre['default'].keys()):
            # create a gensound line object with the starting and ending values for each parameter based on the mood found from timbre.json
            if (param[:2] == 'lfo') and (param[-5:]=='_type'):
                curves[param] = curves[param] | Constant(lfo[timbre[str(initial_mood)][param]], duration)
            if (param[:5] == 'filter') and (param[-5:]=='_type'):
                curves[param] = curves[param] | Constant(filters[timbre[str(initial_mood)][param]], duration)
            if param[-5:] == '_wave':
                curves[param] = curves[param] | Constant(osc[timbre[str(initial_mood)][param]], duration)
            if param == 'automation':
                pass
            else:
                curves[param] = curves[param] | Line(timbre[str(final_mood)][param], timbre[str(final_mood)][param], duration)
            
#     ic(interval)
#     ic(initial_mood, final_mood)
# ic(curves)

In [ ]:
for i in range(len(output['chords'])):
    if i == 0:
        audio = 

## Debugging

In [168]:
import main
ams = AmbientMusicSynthesis()

NameError: name 'AmbientMusicSynthesis' is not defined

In [16]:
o1 = Sawtooth('A3',5e3)
o2 = Sawtooth('A4',5e3)

out = o1 + o2

filter1 = synth.filter1
filter2 = synth.filter2
out *= filter1
out *= filter2
out *= ADSR(attack=synth.attack, decay=synth.decay, sustain=synth.sustain, release=synth.release)
out.play()

In [28]:
def detuned_array(pitch, duration, width, amount):
  # amount = how many oscillators in the array
  # width = the difference in cents between the highest and lowest oscillators in the array
  all_cents = [i*width/amount - width/2 for i in range(amount)] # how much to detune each signal in the array
  return gensound.mix([ Sawtooth(f"{pitch}{round(cents):+}", duration) for cents in all_cents ])

# detuned_array("D4", duration=5e3, width=0, amount=1).play() # just one
# detuned_array("E4", duration=5e3, width=10, amount=3).play() # three, across 10 cents
detuned_array("F#4", duration=5e3, width=40, amount=3).play() # many

In [ ]:
# n_voices = how many oscillators in the array
# detune_range = the difference in cents between the highest and lowest oscillators in the array
if isinstance(pitch, str):
    freq = self.pitch_to_freq(self.NoteToMidi(pitch))
detune_range = freq * int(detune_pct)/100
all_cents = [i*detune_range/n_voices - detune_range/2 for i in range(n_voices)] # how much to detune each signal in the array
# print(isinstance(pitch, str))
match wave_type: # REQUIRES PYTHON 3.10
    case 'sine':
        f = Sine
    case 'square':
        f = Square
    case 'triangle':
        f = Triangle
    case 'sawtooth':
        f = Sawtooth
if isinstance(pitch, str):
    freq = self.pitch_to_freq(self.NoteToMidi(pitch))
return gensound.mix([f(self.shift_freq_by_cents(freq*relative_shift,cents),duration) for cents in all_cents])

In [17]:
out.play()

In [ ]:
def octave_to_multiplier(octave):
    return 2**(octave-4)

In [16]:
pitch = 'A3'
relative_shift = 2
duration = 2e3
detune_pct = 8
n_voices = 4
wave_type='sawtooth'
signal = synth.detuned_voices(pitch, relative_shift, duration, detune_pct, n_voices, wave_type)
signal.play()

In [21]:
osc1 = synth.osc1
osc2 = synth.osc2
# filter1 = synth.filter1
# filter2 = synth.filter2

output = osc1 + osc2 
# output *= filter1
# output *= filter2
output *= ADSR(attack=synth.attack, decay=synth.decay, sustain=synth.sustain, release=synth.release)
output.play()

In [169]:
np.linspace(1, 10, 5, dtype=int)

array([ 1,  3,  5,  7, 10])

In [6]:
synth.osc1.play()

In [46]:
filters={}
for filter_number in ['filter1','filter2','filter3']:
    if mellow_o[filter_number+'_type'] == 'na':
        mellow_o[filter_number] = None
    else:
        match mellow_o[filter_number+'_type']:
            case 'lowpass':
                filters[filter_number] = SimpleLPF
            case 'highpass':
                filters[filter_number] = SimpleHPF
            case 'bandpass':
                filters[filter_number] = SimpleBandPass
            case 'lowshelf':
                filters[filter_number] = SimpleLowShelf
            case 'highshelf':
                filters[filter_number] = SimpleHighShelf